In [1]:
! pip install torch torchvision torchaudio
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
! pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

Looking in indexes: https://download.pytorch.org/whl/nightly/cu121


In [2]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100.0%


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
    

In [4]:
# sample execution (requires torchvision)

filename = "q.jpg"

from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-2.4771e+00, -4.8366e+00, -1.4751e+00, -2.2313e+00, -2.5577e+00,
         5.6699e-01,  3.7114e+00, -2.8442e+00,  9.1893e-02, -8.0581e-01,
         1.1394e+00, -2.1551e+00, -2.0182e+00, -2.7192e+00, -2.5532e+00,
         1.1272e+00, -3.9067e+00,  5.2456e-01, -2.8941e-01, -3.5880e+00,
        -2.0681e+00, -2.7790e+00, -5.5623e+00, -4.6825e+00,  4.4137e-01,
        -3.8956e-01, -1.2232e+00, -1.5931e+00,  9.9696e-02, -1.4310e+00,
        -3.5503e+00, -3.7994e+00, -2.6149e+00, -1.6377e+00, -3.4256e-01,
        -9.2095e-01,  3.0366e+00, -8.1366e-01, -4.8081e+00, -3.6392e-01,
        -5.4408e+00, -4.3032e+00, -5.4705e+00, -2.5044e+00, -4.4041e+00,
        -5.3792e-01, -6.8355e+00, -3.7440e+00, -2.1419e+00, -4.6452e-01,
        -9.0534e-01,  3.3675e+00, -4.4637e+00, -2.9865e+00, -1.7963e+00,
        -3.1361e+00, -4.4541e+00, -1.5139e+00, -4.5803e+00, -4.3969e+00,
        -2.0942e+00, -1.9690e-01,  5.1578e-01, -3.1738e+00, -3.8564e+00,
        -2.1530e+00, -1.3796e+00, -3.5523e+00, -3.6

In [5]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2024-06-22 02:46:59--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2024-06-22 02:47:00 (11.9 MB/s) - ‘imagenet_classes.txt.1’ saved [10472/10472]



In [6]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 10)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())
    

German shepherd 0.38775381445884705
malinois 0.3156793713569641
muzzle 0.07311662286520004
kelpie 0.0688679963350296
Border terrier 0.032726917415857315
wallaby 0.02662491798400879
bluetick 0.014497254975140095
Rottweiler 0.013445338234305382
Norwegian elkhound 0.011689859442412853
miniature pinscher 0.009485425427556038
